# CNN and Tensorflow based face detection
This code uses OpenCV and TensorFlow to perform real-time face detection and recognition. It loads a pre-trained face detection model and a Haar cascade classifier for face detection. It captures frames from the camera, detects faces in each frame using the cascade classifier, and then uses a pre-trained CNN model to recognize the detected faces. Bounding boxes and labels are drawn on the frames to indicate the detected faces. The code also calculates precision, recall, and accuracy metrics based on the ground truth labels and predicted labels. Finally, the processed frames with bounding boxes and labels are displayed in real-time.

In [61]:
import cv2
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load the pre-trained face detection model
model = tf.keras.models.load_model('/Users/nikhil/Downloads/Face Recognition System/keras_model.h5')

# Load the Haar cascade for face detection
face_cascade = cv2.CascadeClassifier('/Users/nikhil/Downloads/Face Recognition System/haarcascade_frontalface_default.xml')

# Initialize the video capture
video_capture = cv2.VideoCapture(0)

# Initialize variables for accuracy metrics
ground_truth_labels = []
predicted_labels = []

while True:
    # Read frame from the video capture
    ret, frame = video_capture.read()

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform face detection using Haar cascade
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the face region from the frame
        face = frame[y:y+h, x:x+w]

        # Preprocess the face image for the CNN model
        face = cv2.resize(face, (224, 224), interpolation=cv2.INTER_LINEAR)
        face = face / 255.0
        face = np.expand_dims(face, axis=0)

        # Make predictions using the CNN model
        predictions = model.predict(face)
        class_index = np.argmax(predictions[0])
        class_label = ['Not a Face', 'Face'][class_index]
        confidence = predictions[0][class_index]

        # Draw bounding box and label on the frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f'{class_label} ({confidence:.2f})', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Append the ground truth and predicted labels
        ground_truth_labels.append(1)  # Assuming all detected faces as ground truth
        predicted_labels.append(class_index)

    # Display the resulting frame
    cv2.imshow('Face Detection', frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Calculate accuracy metrics
ground_truth_labels = np.array(ground_truth_labels)
predicted_labels = np.array(predicted_labels)

precision = precision_score(ground_truth_labels, predicted_labels)
recall = recall_score(ground_truth_labels, predicted_labels)
accuracy = accuracy_score(ground_truth_labels, predicted_labels)

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'Accuracy: {accuracy:.2f}')

# Release the video capture and close the OpenCV windows
video_capture.release()
cv2.destroyAllWindows()

[ WARN:0@6409.114] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


1/1 [==============================] - 0s 16ms/step
Precision: 1.00
Recall: 0.56
Accuracy: 0.56
